<a href="https://colab.research.google.com/gist/maclandrol/e3340c2a682583a24895c0e34d09e8da/13.2_MedSAM_Segmentation_Interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentation d'Images Médicales avec MedSAM pour Étudiants en Médecine

## Introduction pour les Étudiants en Médecine

Bienvenue dans ce tutoriel d'intelligence artificielle appliquée à l'imagerie médicale ! En tant que futurs médecins, vous allez découvrir comment l'IA peut assister dans l'analyse d'images médicales pour améliorer le diagnostic et le suivi des patients.

### Qu'est-ce que MedSAM ?

**MedSAM** (Medical Segment Anything Model) est une version spécialement adaptée du modèle SAM de Meta AI pour le domaine médical. Il permet de :
- **Segmenter automatiquement** des structures anatomiques ou pathologiques
- **Délimiter précisément** des régions d'intérêt (tumeurs, organes, lésions)
- **Assister les radiologues** dans leur travail quotidien

### Applications Cliniques :
- 🩻 **Radiologie** : Délimitation de tumeurs en imagerie
- 🫀 **Cardiologie** : Analyse des cavités cardiaques
- 🧠 **Neurologie** : Segmentation de structures cérébrales
- 👁️ **Ophtalmologie** : Analyse de la rétine

### Objectifs Pédagogiques :
1. Comprendre le principe de la segmentation d'images médicales
2. Utiliser MedSAM pour analyser des images de cancer du sein
3. Évaluer la performance du modèle
4. Tester le modèle sur vos propres images

---

## Configuration de l'Environnement Google Colab

In [ ]:
# Vérification de l'environnement Google Colab et configuration GPU
import sys
import torch

# Vérification de l'environnement
if 'google.colab' in sys.modules:
    print("✅ Environnement Google Colab détecté")
    IN_COLAB = True
else:
    print("⚠️ Environnement local détecté")
    IN_COLAB = False

# Vérification du GPU
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = "cpu"
    print("⚠️ GPU non disponible, utilisation du CPU (plus lent)")
    
print(f"\n🔧 Dispositif utilisé: {device}")

## Installation des Bibliothèques Spécialisées

Nous installons les outils nécessaires pour l'analyse d'images médicales :

In [ ]:
# Installation des bibliothèques spécialisées pour l'IA médicale
print("📦 Installation des bibliothèques spécialisées...")

# Installation de Transformers (version récente pour MedSAM)
!pip install -q git+https://github.com/huggingface/transformers.git

# Installation des bibliothèques pour les datasets médicaux
!pip install -q datasets

# Bibliothèques pour l'analyse quantitative médicale
!pip install -q scikit-image matplotlib seaborn

print("✅ Installation terminée !")

## Montage de Google Drive (pour sauvegarder vos résultats)

Nous montons votre Google Drive pour pouvoir sauvegarder les analyses :

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive monté avec succès !")
    
    # Création du dossier pour vos analyses médicales
    import os
    results_dir = '/content/drive/MyDrive/AnalysesMedicalesIA/'
    os.makedirs(results_dir, exist_ok=True)
    print(f"📁 Dossier créé : {results_dir}")
else:
    results_dir = './resultats_analyses/'
    import os
    os.makedirs(results_dir, exist_ok=True)
    print(f"📁 Dossier local créé : {results_dir}")

## Chargement du Modèle MedSAM

MedSAM est un modèle spécialement entraîné pour analyser des images médicales. Il a été développé par l'équipe du Dr. Bo Wang et améliore significativement les performances par rapport aux modèles génériques.

In [ ]:
from transformers import SamModel, SamProcessor
import torch

print("🏥 Chargement du modèle MedSAM spécialisé...")
print("Ce modèle a été entraîné sur des milliers d'images médicales !")

# Chargement du processeur (prépare les images pour le modèle)
processor = SamProcessor.from_pretrained("wanglab/medsam-vit-base")
print("✅ Processeur d'images médicales chargé")

# Chargement du modèle MedSAM
model = SamModel.from_pretrained("wanglab/medsam-vit-base").to(device)
print("✅ Modèle MedSAM chargé et configuré")

print(f"\n🔬 Le modèle est prêt à analyser des images médicales sur {device}")

## Chargement des Données Médicales : Cancer du Sein

Nous utilisons un dataset d'images échographiques de cancer du sein. Ces images proviennent d'examens réels et ont été annotées par des spécialistes.

In [ ]:
from datasets import load_dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

print("🩻 Chargement du dataset d'échographies mammaires...")

# Chargement du dataset de cancer du sein
dataset = load_dataset("nielsr/breast-cancer", split="train")
print(f"✅ Dataset chargé : {len(dataset)} images d'échographies mammaires")

# Information sur le dataset pour les étudiants
print("\n📊 Informations sur le dataset :")
print(f"- Nombre d'images : {len(dataset)}")
print(f"- Type d'imagerie : Échographie mammaire")
print(f"- Annotations : Segmentations des lésions par des radiologues")

# Sélection d'un exemple représentatif
idx = 10  # Image particulièrement intéressante pour l'enseignement

# Chargement de l'image
image = dataset[idx]["image"]
print(f"\n🔍 Image sélectionnée : #{idx}")
print(f"Taille de l'image : {image.size}")

## Visualisation de l'Image Échographique

Examinons l'image échographique comme le ferait un radiologue :

In [ ]:
# Affichage de l'image échographique
plt.figure(figsize=(10, 8))
plt.imshow(image, cmap='gray')
plt.title("Échographie Mammaire - Image Originale\n(Recherchez les zones suspectes)", fontsize=14, fontweight='bold')
plt.axis('on')
plt.grid(True, alpha=0.3)

# Ajout d'annotations pédagogiques
plt.xlabel("Largeur (pixels)", fontsize=12)
plt.ylabel("Hauteur (pixels)", fontsize=12)

print("\n🏥 Points d'observation clinique :")
print("- Recherchez les zones hypoéchogènes (plus sombres)")
print("- Observez les contours et la forme des structures")
print("- Notez l'homogénéité ou l'hétérogénéité des tissus")

plt.tight_layout()
plt.show()

## Analyse de la Segmentation de Référence

Voyons maintenant la segmentation réalisée par les radiologues experts :

In [ ]:
# Chargement de la segmentation de référence (ground truth)
ground_truth_seg = np.array(dataset[idx]["label"])

print("🎯 Analyse de la segmentation de référence :")
print(f"Valeurs uniques dans la segmentation : {np.unique(ground_truth_seg)}")
print("0 = Tissu normal, 1 = Lésion/Structure d'intérêt")

# Calcul de statistiques médicales
total_pixels = ground_truth_seg.size
lesion_pixels = np.sum(ground_truth_seg == 1)
normal_pixels = np.sum(ground_truth_seg == 0)
lesion_percentage = (lesion_pixels / total_pixels) * 100

print(f"\n📊 Statistiques quantitatives :")
print(f"- Pixels totaux : {total_pixels:,}")
print(f"- Pixels de lésion : {lesion_pixels:,}")
print(f"- Pixels normaux : {normal_pixels:,}")
print(f"- Pourcentage de lésion : {lesion_percentage:.2f}%")

# Visualisation de la segmentation avec palette médicale
palette = [[120, 120, 120], [255, 100, 100]]  # Gris pour normal, Rouge pour lésion
color_seg = np.zeros((ground_truth_seg.shape[0], ground_truth_seg.shape[1], 3), dtype=np.uint8)

for label, color in enumerate(palette):
    color_seg[ground_truth_seg == label, :] = color

plt.figure(figsize=(10, 8))
plt.imshow(color_seg)
plt.title("Segmentation de Référence par les Radiologues\n(Rouge = Lésion, Gris = Tissu Normal)", 
          fontsize=14, fontweight='bold')
plt.axis('on')
plt.grid(True, alpha=0.3)

# Ajout d'une légende médicale
import matplotlib.patches as mpatches
normal_patch = mpatches.Patch(color=[120/255, 120/255, 120/255], label='Tissu Normal')
lesion_patch = mpatches.Patch(color=[1, 100/255, 100/255], label='Lésion Identifiée')
plt.legend(handles=[normal_patch, lesion_patch], loc='upper right')

plt.tight_layout()
plt.show()

## Génération Automatique de la Boîte Englobante

Pour utiliser MedSAM, nous devons définir une région d'intérêt. En pratique clinique, cette région serait indiquée par le radiologue.

In [ ]:
def get_bounding_box(ground_truth_map, perturbation=True):
    """
    Génère une boîte englobante autour de la lésion
    
    En pratique clinique, cette boîte serait dessinée par le radiologue
    sur l'interface du logiciel d'imagerie médicale.
    """
    # Recherche des coordonnées de la lésion
    y_indices, x_indices = np.where(ground_truth_map > 0)
    
    if len(x_indices) == 0 or len(y_indices) == 0:
        print("⚠️ Aucune lésion détectée dans cette image")
        return None
    
    # Calcul des bornes
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    
    # Simulation de l'imprécision humaine (optionnel)
    if perturbation:
        H, W = ground_truth_map.shape
        x_min = max(0, x_min - np.random.randint(0, 20))
        x_max = min(W, x_max + np.random.randint(0, 20))
        y_min = max(0, y_min - np.random.randint(0, 20))
        y_max = min(H, y_max + np.random.randint(0, 20))
    
    bbox = [x_min, y_min, x_max, y_max]
    
    # Calculs médicaux
    width = x_max - x_min
    height = y_max - y_min
    area = width * height
    
    print(f"📏 Dimensions de la région d'intérêt :")
    print(f"- Largeur : {width} pixels")
    print(f"- Hauteur : {height} pixels")
    print(f"- Surface : {area} pixels²")
    
    return bbox

# Génération de la boîte englobante
input_boxes = get_bounding_box(ground_truth_seg)
print(f"\n🎯 Coordonnées de la boîte : {input_boxes}")
print("Format : [x_min, y_min, x_max, y_max]")

## Visualisation de la Région d'Intérêt

Affichons la boîte englobante sur l'image originale :

In [ ]:
def show_box(box, ax, color='green', label='Région d\'Intérêt'):
    """Affiche une boîte englobante sur l'image"""
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, 
                              edgecolor=color, 
                              facecolor=(0,0,0,0), 
                              linewidth=3,
                              label=label))

def show_boxes_on_image(raw_image, boxes, title="Image avec Région d'Intérêt"):
    """Affiche l'image avec les boîtes englobantes"""
    plt.figure(figsize=(12, 10))
    plt.imshow(raw_image, cmap='gray')
    
    for i, box in enumerate(boxes):
        show_box(box, plt.gca(), color='lime', label=f'ROI {i+1}')
    
    plt.title(title + "\n(Cadre vert = Zone à analyser par MedSAM)", 
              fontsize=14, fontweight='bold')
    plt.legend()
    plt.axis('on')
    plt.grid(True, alpha=0.3)
    
    # Annotations cliniques
    plt.xlabel("Position horizontale (pixels)", fontsize=12)
    plt.ylabel("Position verticale (pixels)", fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Affichage de l'image avec la région d'intérêt
show_boxes_on_image(image, [input_boxes], "Échographie avec Région d'Intérêt Définie")

print("\n🏥 Interprétation clinique :")
print("- Le cadre vert indique la zone que MedSAM va analyser")
print("- Cette zone correspond à la région suspecte identifiée")
print("- En pratique, cette zone serait sélectionnée par le radiologue")

## Traitement par le Modèle MedSAM

Maintenant, laissons MedSAM analyser l'image et prédire la segmentation :

In [ ]:
import torch

print("🔬 Préparation de l'image pour l'analyse par MedSAM...")

# Préparation des données d'entrée pour le modèle
inputs = processor(image, input_boxes=[input_boxes], return_tensors="pt").to(device)

print("📊 Dimensions des données d'entrée :")
for k, v in inputs.items():
    print(f"- {k}: {v.shape}")

print(f"\n🧠 Traitement par le modèle MedSAM...")

# Passage du modèle vers le bon dispositif
model.to(device)

# Analyse par MedSAM (sans calcul de gradients pour l'inférence)
with torch.no_grad():
    outputs = model(**inputs, multimask_output=False)

print(f"✅ Analyse terminée !")
print(f"📏 Dimensions du masque prédit : {outputs.pred_masks.shape}")
print("Format : [batch, masques, canaux, hauteur, largeur]")

## Post-traitement Spécifique à MedSAM

MedSAM utilise une fonction de perte spécialisée (DiceWithSigmoid), nous devons appliquer le post-traitement approprié :

In [ ]:
print("⚙️ Post-traitement spécialisé pour l'imagerie médicale...")

# Application de la fonction sigmoïde (normalisation entre 0 et 1)
medsam_seg_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
print(f"✅ Probabilités normalisées (0-1)")

# Conversion en numpy pour l'analyse
medsam_seg_prob = medsam_seg_prob.cpu().numpy().squeeze()

# Conversion en masque binaire (seuil à 0.5)
medsam_seg = (medsam_seg_prob > 0.5).astype(np.uint8)

print(f"📊 Statistiques de la prédiction :")
print(f"- Probabilité moyenne : {np.mean(medsam_seg_prob):.3f}")
print(f"- Probabilité maximale : {np.max(medsam_seg_prob):.3f}")
print(f"- Probabilité minimale : {np.min(medsam_seg_prob):.3f}")
print(f"- Pixels classés comme lésion : {np.sum(medsam_seg)} / {medsam_seg.size}")
print(f"- Pourcentage de lésion prédit : {(np.sum(medsam_seg)/medsam_seg.size)*100:.2f}%")

## Visualisation des Résultats de MedSAM

Comparons la prédiction de MedSAM avec la segmentation de référence :

In [ ]:
def show_mask(mask, ax, color_rgba=[30/255, 144/255, 255/255, 0.6], title=""):
    """Affiche un masque de segmentation avec transparence"""
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * np.array(color_rgba).reshape(1, 1, -1)
    ax.imshow(mask_image)
    ax.set_title(title, fontsize=12, fontweight='bold')

# Création d'une figure comparative
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle("Analyse Comparative : MedSAM vs Référence Clinique", fontsize=16, fontweight='bold')

# 1. Image originale
axes[0,0].imshow(np.array(image), cmap='gray')
axes[0,0].set_title("Image Échographique Originale", fontweight='bold')
axes[0,0].axis('off')

# 2. Prédiction MedSAM
axes[0,1].imshow(np.array(image), cmap='gray')
show_mask(medsam_seg, axes[0,1], color_rgba=[0, 1, 0, 0.6])  # Vert
axes[0,1].set_title("Prédiction MedSAM\n(Masque Vert)", fontweight='bold')
axes[0,1].axis('off')

# 3. Segmentation de référence
axes[1,0].imshow(np.array(image), cmap='gray')
show_mask(ground_truth_seg, axes[1,0], color_rgba=[1, 0, 0, 0.6])  # Rouge
axes[1,0].set_title("Segmentation de Référence\n(Masque Rouge)", fontweight='bold')
axes[1,0].axis('off')

# 4. Comparaison superposée
axes[1,1].imshow(np.array(image), cmap='gray')
show_mask(ground_truth_seg, axes[1,1], color_rgba=[1, 0, 0, 0.4])  # Rouge transparent
show_mask(medsam_seg, axes[1,1], color_rgba=[0, 1, 0, 0.4])      # Vert transparent
axes[1,1].set_title("Superposition\n(Rouge=Référence, Vert=MedSAM)", fontweight='bold')
axes[1,1].axis('off')

plt.tight_layout()
plt.show()

print("\n🏥 Interprétation clinique :")
print("- Le masque vert montre la prédiction de MedSAM")
print("- Le masque rouge montre la segmentation de référence")
print("- La superposition permet d'évaluer la concordance")
print("- Les zones jaunes (vert+rouge) indiquent un accord parfait")

## Évaluation Quantitative des Performances

Calculons des métriques médicales pour évaluer la performance de MedSAM :

In [ ]:
def calculate_medical_metrics(pred_mask, true_mask):
    """
    Calcule les métriques médicales standard pour évaluer la segmentation
    """
    # Conversion en booléens pour les calculs
    pred = pred_mask.astype(bool)
    true = true_mask.astype(bool)
    
    # Calculs des métriques de base
    intersection = np.logical_and(pred, true)
    union = np.logical_or(pred, true)
    
    # True Positives, False Positives, False Negatives
    tp = np.sum(intersection)
    fp = np.sum(pred) - tp
    fn = np.sum(true) - tp
    tn = np.sum(~union)
    
    # Métriques médicales
    # Coefficient de Dice (F1-score) - très important en imagerie médicale
    dice_coeff = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
    
    # Intersection over Union (IoU) - Jaccard Index
    iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
    
    # Sensibilité (Rappel) - capacité à détecter les lésions
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # Spécificité - capacité à identifier les tissus sains
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # Précision - fiabilité des détections positives
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    # Exactitude globale
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    return {
        'dice_coefficient': dice_coeff,
        'iou': iou,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'accuracy': accuracy,
        'true_positives': tp,
        'false_positives': fp,
        'false_negatives': fn,
        'true_negatives': tn
    }

# Calcul des métriques
metrics = calculate_medical_metrics(medsam_seg, ground_truth_seg)

print("📊 ÉVALUATION QUANTITATIVE DE MEDSAM")
print("=" * 50)
print(f"\n🎯 Métriques de Performance :")
print(f"- Coefficient de Dice : {metrics['dice_coefficient']:.3f} (0=mauvais, 1=parfait)")
print(f"- IoU (Jaccard) : {metrics['iou']:.3f} (0=mauvais, 1=parfait)")
print(f"- Exactitude : {metrics['accuracy']:.3f} ({metrics['accuracy']*100:.1f}%)")

print(f"\n🩺 Métriques Cliniques :")
print(f"- Sensibilité : {metrics['sensitivity']:.3f} (détection des lésions)")
print(f"- Spécificité : {metrics['specificity']:.3f} (identification tissu sain)")
print(f"- Précision : {metrics['precision']:.3f} (fiabilité des détections)")

print(f"\n🔢 Détail des Prédictions :")
print(f"- Vrais Positifs : {metrics['true_positives']} pixels")
print(f"- Faux Positifs : {metrics['false_positives']} pixels")
print(f"- Faux Négatifs : {metrics['false_negatives']} pixels")
print(f"- Vrais Négatifs : {metrics['true_negatives']} pixels")

# Interprétation clinique
print(f"\n🏥 INTERPRÉTATION CLINIQUE :")
if metrics['dice_coefficient'] > 0.8:
    print("✅ Excellente concordance avec l'expert radiologue")
elif metrics['dice_coefficient'] > 0.6:
    print("👍 Bonne concordance avec l'expert radiologue")
elif metrics['dice_coefficient'] > 0.4:
    print("⚠️ Concordance modérée - nécessite révision")
else:
    print("❌ Concordance faible - résultat non fiable")

if metrics['sensitivity'] > 0.9:
    print("✅ Excellente détection des lésions")
elif metrics['sensitivity'] < 0.7:
    print("⚠️ Risque de manquer des lésions (faux négatifs)")

if metrics['specificity'] > 0.9:
    print("✅ Excellente identification du tissu sain")
elif metrics['specificity'] < 0.8:
    print("⚠️ Risque de sur-diagnostic (faux positifs)")

## Analyse de Cas Multiples

Testons MedSAM sur plusieurs images pour évaluer sa robustesse :

In [ ]:
def analyze_multiple_cases(dataset, model, processor, device, num_cases=5):
    """
    Analyse plusieurs cas pour évaluer la robustesse du modèle
    """
    results = []
    
    print(f"🔬 Analyse de {num_cases} cas cliniques...")
    
    # Sélection d'indices variés
    indices = np.linspace(0, len(dataset)-1, num_cases, dtype=int)
    
    for i, idx in enumerate(indices):
        print(f"\n📋 Cas #{i+1} (Image #{idx}) :")
        
        # Chargement des données
        image = dataset[idx]["image"]
        ground_truth = np.array(dataset[idx]["label"])
        
        # Génération de la boîte englobante
        bbox = get_bounding_box(ground_truth, perturbation=False)
        
        if bbox is None:
            print("   ⚠️ Pas de lésion dans cette image - cas ignoré")
            continue
        
        # Prédiction MedSAM
        inputs = processor(image, input_boxes=[bbox], return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model(**inputs, multimask_output=False)
        
        # Post-traitement
        seg_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
        prediction = (seg_prob.cpu().numpy().squeeze() > 0.5).astype(np.uint8)
        
        # Calcul des métriques
        metrics = calculate_medical_metrics(prediction, ground_truth)
        
        results.append({
            'case_id': idx,
            'dice': metrics['dice_coefficient'],
            'iou': metrics['iou'],
            'sensitivity': metrics['sensitivity'],
            'specificity': metrics['specificity']
        })
        
        print(f"   Dice: {metrics['dice_coefficient']:.3f}, IoU: {metrics['iou']:.3f}")
        print(f"   Sensibilité: {metrics['sensitivity']:.3f}, Spécificité: {metrics['specificity']:.3f}")
    
    return results

# Analyse de plusieurs cas
multi_results = analyze_multiple_cases(dataset, model, processor, device, num_cases=5)

if multi_results:
    # Calcul des statistiques globales
    mean_dice = np.mean([r['dice'] for r in multi_results])
    std_dice = np.std([r['dice'] for r in multi_results])
    mean_sensitivity = np.mean([r['sensitivity'] for r in multi_results])
    mean_specificity = np.mean([r['specificity'] for r in multi_results])
    
    print(f"\n📊 PERFORMANCE GLOBALE SUR {len(multi_results)} CAS :")
    print(f"=" * 50)
    print(f"Coefficient de Dice moyen : {mean_dice:.3f} ± {std_dice:.3f}")
    print(f"Sensibilité moyenne : {mean_sensitivity:.3f}")
    print(f"Spécificité moyenne : {mean_specificity:.3f}")
    
    # Visualisation des performances
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    dice_scores = [r['dice'] for r in multi_results]
    plt.bar(range(len(dice_scores)), dice_scores, color='skyblue', edgecolor='navy')
    plt.axhline(y=mean_dice, color='red', linestyle='--', label=f'Moyenne: {mean_dice:.3f}')
    plt.title('Coefficient de Dice par Cas')
    plt.xlabel('Numéro du Cas')
    plt.ylabel('Score de Dice')
    plt.legend()
    plt.ylim(0, 1)
    
    plt.subplot(1, 2, 2)
    metrics_names = ['Dice', 'IoU', 'Sensibilité', 'Spécificité']
    metrics_values = [
        np.mean([r['dice'] for r in multi_results]),
        np.mean([r['iou'] for r in multi_results]),
        np.mean([r['sensitivity'] for r in multi_results]),
        np.mean([r['specificity'] for r in multi_results])
    ]
    
    colors = ['lightcoral', 'lightblue', 'lightgreen', 'gold']
    plt.bar(metrics_names, metrics_values, color=colors, edgecolor='black')
    plt.title('Performance Moyenne de MedSAM')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    
    # Ajout des valeurs sur les barres
    for i, v in enumerate(metrics_values):
        plt.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## Fonction d'Upload et de Test Personnalisé

Testez MedSAM sur vos propres images médicales !

In [ ]:
from google.colab import files
import io
from PIL import Image

def test_custom_image():
    """
    Permet de tester MedSAM sur une image personnalisée
    """
    print("📁 Upload de votre image médicale...")
    print("Formats supportés : JPG, PNG, TIFF")
    print("Recommandations : Images en niveaux de gris, résolution 256x256 à 1024x1024")
    
    if IN_COLAB:
        # Upload dans Google Colab
        uploaded = files.upload()
        
        if not uploaded:
            print("❌ Aucun fichier uploadé")
            return None
        
        # Traitement du premier fichier uploadé
        filename = list(uploaded.keys())[0]
        print(f"✅ Fichier reçu : {filename}")
        
        # Chargement de l'image
        image_bytes = uploaded[filename]
        image = Image.open(io.BytesIO(image_bytes))
        
    else:
        # Mode local - demande le chemin
        import tkinter as tk
        from tkinter import filedialog
        root = tk.Tk()
        root.withdraw()
        
        file_path = filedialog.askopenfilename(
            title="Sélectionnez votre image médicale",
            filetypes=[("Images", "*.png *.jpg *.jpeg *.tiff *.bmp")]
        )
        
        if not file_path:
            print("❌ Aucun fichier sélectionné")
            return None
            
        image = Image.open(file_path)
        filename = file_path.split('/')[-1]
    
    # Conversion en niveaux de gris si nécessaire
    if image.mode != 'L':
        print("🔄 Conversion en niveaux de gris...")
        image = image.convert('L')
    
    print(f"📏 Dimensions de l'image : {image.size}")
    
    # Redimensionnement si trop grande
    max_size = 1024
    if max(image.size) > max_size:
        print(f"🔄 Redimensionnement (max {max_size}px)...")
        image.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
        print(f"📏 Nouvelles dimensions : {image.size}")
    
    return image, filename

def interactive_roi_selection(image):
    """
    Interface simple pour sélectionner une région d'intérêt
    """
    print("\n🎯 Sélection de la région d'intérêt :")
    print("Cliquez pour définir le coin supérieur gauche, puis le coin inférieur droit")
    
    # Affichage de l'image pour sélection
    plt.figure(figsize=(10, 8))
    plt.imshow(image, cmap='gray')
    plt.title("Cliquez pour définir la région d'intérêt\n(Coin sup. gauche puis coin inf. droit)")
    plt.axis('on')
    
    # Récupération des clics (simulation - en réalité utilisez ginput)
    coords = plt.ginput(2, timeout=30, show_clicks=True)
    
    if len(coords) < 2:
        print("❌ Sélection incomplète, utilisation de l'image entière")
        return [0, 0, image.width, image.height]
    
    x1, y1 = int(coords[0][0]), int(coords[0][1])
    x2, y2 = int(coords[1][0]), int(coords[1][1])
    
    # Assurer l'ordre correct
    x_min, x_max = min(x1, x2), max(x1, x2)
    y_min, y_max = min(y1, y2), max(y1, y2)
    
    plt.close()
    
    print(f"✅ ROI sélectionnée : [{x_min}, {y_min}, {x_max}, {y_max}]")
    return [x_min, y_min, x_max, y_max]

def analyze_custom_image(image, bbox, filename):
    """
    Analyse une image personnalisée avec MedSAM
    """
    print(f"\n🔬 Analyse de {filename} avec MedSAM...")
    
    # Préparation pour MedSAM
    inputs = processor(image, input_boxes=[bbox], return_tensors="pt").to(device)
    
    # Prédiction
    with torch.no_grad():
        outputs = model(**inputs, multimask_output=False)
    
    # Post-traitement
    seg_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
    prediction = (seg_prob.cpu().numpy().squeeze() > 0.5).astype(np.uint8)
    probabilities = seg_prob.cpu().numpy().squeeze()
    
    # Statistiques
    total_pixels = prediction.size
    positive_pixels = np.sum(prediction)
    positive_percentage = (positive_pixels / total_pixels) * 100
    max_confidence = np.max(probabilities)
    avg_confidence = np.mean(probabilities[prediction == 1]) if positive_pixels > 0 else 0
    
    print(f"📊 Résultats de l'analyse :")
    print(f"- Pixels détectés : {positive_pixels} / {total_pixels}")
    print(f"- Pourcentage de lésion : {positive_percentage:.2f}%")
    print(f"- Confiance maximale : {max_confidence:.3f}")
    print(f"- Confiance moyenne (zones positives) : {avg_confidence:.3f}")
    
    # Visualisation
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(f"Analyse MedSAM : {filename}", fontsize=16, fontweight='bold')
    
    # Image originale avec ROI
    axes[0].imshow(image, cmap='gray')
    show_box(bbox, axes[0], color='lime')
    axes[0].set_title("Image Originale + ROI")
    axes[0].axis('off')
    
    # Carte de probabilités
    im1 = axes[1].imshow(probabilities, cmap='hot', vmin=0, vmax=1)
    axes[1].set_title("Carte de Probabilités")
    axes[1].axis('off')
    plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
    
    # Segmentation finale
    axes[2].imshow(image, cmap='gray')
    show_mask(prediction, axes[2], color_rgba=[0, 1, 0, 0.6])
    axes[2].set_title("Segmentation Prédite")
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Sauvegarde des résultats
    if IN_COLAB:
        save_path = f"{results_dir}analyse_{filename.split('.')[0]}.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"💾 Résultats sauvegardés : {save_path}")
    
    return prediction, probabilities

# Interface pour tester une image personnalisée
print("🏥 TEST SUR IMAGE PERSONNALISÉE")
print("=" * 40)
print("Vous pouvez maintenant tester MedSAM sur vos propres images médicales !")
print("\nPour commencer, exécutez les cellules suivantes :")
print("1. Upload de votre image")
print("2. Sélection de la région d'intérêt")
print("3. Analyse automatique par MedSAM")

In [ ]:
# CELLULE D'UPLOAD - Exécutez pour tester votre image
custom_image, custom_filename = test_custom_image()

In [ ]:
# CELLULE DE SÉLECTION ROI
if 'custom_image' in locals() and custom_image is not None:
    # Affichage pour sélection manuelle ou automatique
    print("Choisissez le mode de sélection :")
    print("1. Manuel (recommandé) : Vous cliquez sur l'image")
    print("2. Automatique : Analyse de l'image entière")
    
    mode = input("Tapez 1 ou 2 : ").strip()
    
    if mode == "1":
        custom_bbox = interactive_roi_selection(custom_image)
    else:
        # Mode automatique - toute l'image
        custom_bbox = [0, 0, custom_image.width, custom_image.height]
        print(f"✅ ROI automatique : toute l'image {custom_image.size}")
else:
    print("❌ Pas d'image chargée. Exécutez d'abord la cellule d'upload.")

In [ ]:
# CELLULE D'ANALYSE
if 'custom_image' in locals() and 'custom_bbox' in locals():
    custom_prediction, custom_probabilities = analyze_custom_image(
        custom_image, custom_bbox, custom_filename
    )
    
    print("\n✅ Analyse terminée !")
    print("\n🏥 Recommandations cliniques :")
    print("- Ces résultats sont à des fins éducatives uniquement")
    print("- Toujours consulter un radiologue qualifié pour le diagnostic")
    print("- L'IA est un outil d'assistance, pas de remplacement")
    print("- Corréler avec la clinique et autres examens")
else:
    print("❌ Pas d'image ou de ROI définie. Exécutez les cellules précédentes.")

## Sauvegarde et Documentation de vos Analyses

Créons un rapport de vos analyses pour révision :

In [ ]:
import datetime
import json

def create_analysis_report(results, custom_results=None):
    """
    Crée un rapport d'analyse pour documentation
    """
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    report = {
        "session_info": {
            "timestamp": timestamp,
            "model": "MedSAM (wanglab/medsam-vit-base)",
            "device": str(device),
            "dataset": "nielsr/breast-cancer"
        },
        "standard_analysis": {
            "num_cases": len(results) if results else 0,
            "performance_metrics": {}
        },
        "custom_analysis": {}
    }
    
    # Métriques sur les cas standards
    if results:
        metrics_summary = {
            "mean_dice": float(np.mean([r['dice'] for r in results])),
            "std_dice": float(np.std([r['dice'] for r in results])),
            "mean_sensitivity": float(np.mean([r['sensitivity'] for r in results])),
            "mean_specificity": float(np.mean([r['specificity'] for r in results]))
        }
        report["standard_analysis"]["performance_metrics"] = metrics_summary
    
    # Résultats personnalisés
    if custom_results:
        report["custom_analysis"] = {
            "filename": custom_results.get("filename", "unknown"),
            "image_size": custom_results.get("image_size", [0, 0]),
            "roi_coordinates": custom_results.get("bbox", [0, 0, 0, 0]),
            "positive_pixels": int(custom_results.get("positive_pixels", 0)),
            "positive_percentage": float(custom_results.get("positive_percentage", 0)),
            "max_confidence": float(custom_results.get("max_confidence", 0))
        }
    
    return report

# Création du rapport
custom_data = None
if 'custom_prediction' in locals():
    custom_data = {
        "filename": custom_filename,
        "image_size": list(custom_image.size),
        "bbox": custom_bbox,
        "positive_pixels": np.sum(custom_prediction),
        "positive_percentage": (np.sum(custom_prediction) / custom_prediction.size) * 100,
        "max_confidence": np.max(custom_probabilities)
    }

analysis_report = create_analysis_report(multi_results, custom_data)

# Sauvegarde du rapport
report_filename = f"rapport_analyse_MedSAM_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
report_path = f"{results_dir}{report_filename}"

with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(analysis_report, f, indent=2, ensure_ascii=False)

print(f"📄 Rapport d'analyse sauvegardé : {report_path}")
print(f"\n📋 Résumé de la session :")
print(f"- Timestamp : {analysis_report['session_info']['timestamp']}")
print(f"- Modèle utilisé : {analysis_report['session_info']['model']}")
print(f"- Device : {analysis_report['session_info']['device']}")
if multi_results:
    print(f"- Cas analysés : {len(multi_results)}")
    print(f"- Performance moyenne : {analysis_report['standard_analysis']['performance_metrics'].get('mean_dice', 0):.3f}")
if custom_data:
    print(f"- Image personnalisée : {custom_data['filename']}")
    print(f"- Détection : {custom_data['positive_percentage']:.2f}% de l'image")

print(f"\n💾 Tous vos résultats sont sauvegardés dans : {results_dir}")

## Considérations Éthiques et Cliniques

### 🏥 Points Importants pour les Futurs Médecins :

#### Responsabilités Professionnelles :
- **L'IA est un outil d'aide au diagnostic**, jamais un remplacement du jugement clinique
- **Toujours corréler** avec l'examen clinique, l'anamnèse et autres examens
- **Maintenir la formation continue** sur les nouvelles technologies d'IA médicale
- **Comprendre les limites** de chaque système d'IA utilisé

#### Éthique Médicale et IA :
- **Consentement éclairé** : Informer les patients de l'utilisation d'IA
- **Transparence** : Expliquer le rôle de l'IA dans le diagnostic
- **Responsabilité** : Le médecin reste responsable du diagnostic final
- **Équité** : S'assurer que l'IA ne génère pas de biais discriminatoires

#### Qualité et Sécurité :
- **Validation clinique** : Utiliser uniquement des systèmes validés cliniquement
- **Formation** : Se former régulièrement sur les outils d'IA utilisés
- **Double vérification** : Toujours vérifier les résultats de l'IA
- **Documentation** : Documenter l'utilisation d'IA dans le dossier patient

### 🔬 Applications Futures en Médecine :

#### Radiologie :
- Détection précoce de cancers
- Analyse quantitative des lésions
- Priorisation des cas urgents

#### Pathologie :
- Analyse histopathologique automatisée
- Gradation tumorale assistée
- Détection de micro-métastases

#### Autres Spécialités :
- Ophtalmologie : Dépistage rétinopathie diabétique
- Dermatologie : Classification des lésions cutanées
- Cardiologie : Analyse échocardiographique

---

## Conclusion

Félicitations ! Vous avez appris à :
1. ✅ **Utiliser MedSAM** pour la segmentation d'images médicales
2. ✅ **Évaluer quantitativement** les performances d'un modèle d'IA
3. ✅ **Tester sur vos propres images** médicales
4. ✅ **Comprendre les implications éthiques** de l'IA en médecine

### 📚 Pour Approfondir :
- Article original MedSAM : [arXiv:2304.12306](https://arxiv.org/abs/2304.12306)
- Repo GitHub : [bowang-lab/MedSAM](https://github.com/bowang-lab/MedSAM)
- Documentation Hugging Face : [transformers/sam](https://huggingface.co/docs/transformers/model_doc/sam)

### 🎓 Message pour les Futurs Médecins :
L'intelligence artificielle transforme rapidement la pratique médicale. En tant que futurs praticiens, vous devez être préparés à utiliser ces outils de manière éthique et efficace. L'IA ne remplacera jamais l'empathie, le jugement clinique et l'expertise médicale humaine, mais elle peut considérablement améliorer la précision diagnostique et l'efficacité des soins.

**Continuez à apprendre, restez curieux, et gardez toujours le patient au centre de vos préoccupations !**